In [2]:
import pulp

# Define the problem
prob = pulp.LpProblem("Maximize_Airline_Revenue", pulp.LpMaximize)

# Decision variables
x_F = pulp.LpVariable('x_F', lowBound=0, cat='Continuous')  # first-class seats
x_B = pulp.LpVariable('x_B', lowBound=0, cat='Continuous')  # business-class seats
x_E = pulp.LpVariable('x_E', lowBound=0, cat='Continuous')  # economy-class seats

# Objective function: Expected revenue
prob += (0.4 * (1600 * x_F + 1100 * x_B + 500 * x_E) +
         0.3 * (1600 * x_F + 1100 * x_B + 500 * x_E) +
         0.3 * (1600 * x_F + 1100 * x_B + 500 * x_E)), "Expected_Revenue"

# Constraints: Ensure the number of seats allocated does not exceed the available space
prob += 2 * x_F + 1.5 * x_B + x_E <= 190, "Space_Constraint"

# Demand constraints: Based on the highest demand scenario
prob += x_F <= 25, "First_Class_Demand"
prob += x_B <= 60, "Business_Class_Demand"
prob += x_E <= 210, "Economy_Class_Demand"

# Solve the problem
prob.solve()

# Output the results
print("Status:", pulp.LpStatus[prob.status])
print("Optimal number of first-class seats:", pulp.value(x_F))
print("Optimal number of business-class seats:", pulp.value(x_B))
print("Optimal number of economy-class seats:", pulp.value(x_E))
print("Maximum Expected Revenue:", pulp.value(prob.objective))


Status: Optimal
Optimal number of first-class seats: 25.0
Optimal number of business-class seats: 60.0
Optimal number of economy-class seats: 50.0
Maximum Expected Revenue: 131000.0


In [9]:
import numpy as np

# Define parameters
num_simulations = 10000
demand_data = {
    "probability": [0.40, 0.30, 0.30],
    "first_class_demand": [25, 12, 5],
    "business_class_demand": [60, 30, 9],
    "economy_class_demand": [210, 170, 150]
}
revenue = {"first_class": 1600, "business_class": 1100, "economy_class": 500}
seat_space = {"first_class": 2, "business_class": 1.5, "economy_class": 1}
total_capacity = 190

# Function to calculate revenue for a given allocation
def calculate_revenue(allocation, demand):
    seats_allocated = {
        "first_class": min(allocation["first_class"], demand["first_class"]),
        "business_class": min(allocation["business_class"], demand["business_class"]),
        "economy_class": min(allocation["economy_class"], demand["economy_class"])
    }
    total_revenue = (
        seats_allocated["first_class"] * revenue["first_class"] +
        seats_allocated["business_class"] * revenue["business_class"] +
        seats_allocated["economy_class"] * revenue["economy_class"]
    )
    return total_revenue

# Monte Carlo simulation
results = []
for _ in range(num_simulations):
    # Randomly select demand scenario
    demand_index = np.random.choice([0, 1, 2], p=demand_data["probability"])
    demand = {
        "first_class": demand_data["first_class_demand"][demand_index],
        "business_class": demand_data["business_class_demand"][demand_index],
        "economy_class": demand_data["economy_class_demand"][demand_index]
    }
    
    # Try different seat allocations to maximize revenue within capacity constraints
    best_revenue = 0
    for first_class_seats in range(0, total_capacity // int(seat_space["first_class"]) + 1):
        for business_class_seats in range(0, total_capacity // int(seat_space["business_class"]) + 1):
            economy_seats = total_capacity - (
                first_class_seats * seat_space["first_class"] +
                business_class_seats * seat_space["business_class"]
            )
            
            if economy_seats >= 0:
                allocation = {
                    "first_class": first_class_seats,
                    "business_class": business_class_seats,
                    "economy_class": int(economy_seats)
                }
                current_revenue = calculate_revenue(allocation, demand)
                
                if current_revenue > best_revenue:
                    best_revenue = current_revenue
                    
    results.append(best_revenue)

# Calculating the expected revenue from simulations
expected_revenue = np.mean(results)
print("Expected Revenue:", expected_revenue)


Expected Revenue: 114165.8


In [1]:
import numpy as np

# Parameters
num_simulations = 10000
total_capacity = 190
demand_data = {
    "probability": [0.40, 0.30, 0.30],
    "first_class_demand": [25, 12, 5],
    "business_class_demand": [60, 30, 9],
    "economy_class_demand": [210, 170, 150]
}
revenue = {"first_class": 1600, "business_class": 1100, "economy_class": 500}
seat_space = {"first_class": 2, "business_class": 1.5, "economy_class": 1}

# Function to calculate revenue for each allocation
def calculate_revenue_per_class(allocation, demand):
    seats_allocated = {
        "first_class": min(allocation["first_class"], demand["first_class"]),
        "business_class": min(allocation["business_class"], demand["business_class"]),
        "economy_class": min(allocation["economy_class"], demand["economy_class"])
    }
    revenue_per_class = {
        "first_class": seats_allocated["first_class"] * revenue["first_class"],
        "business_class": seats_allocated["business_class"] * revenue["business_class"],
        "economy_class": seats_allocated["economy_class"] * revenue["economy_class"]
    }
    total_revenue = sum(revenue_per_class.values())
    return revenue_per_class, total_revenue

# Monte Carlo simulation
first_class_revenues = []
business_class_revenues = []
economy_class_revenues = []
total_revenues = []

for _ in range(num_simulations):
    # Select demand scenario based on probabilities
    demand_index = np.random.choice([0, 1, 2], p=demand_data["probability"])
    demand = {
        "first_class": demand_data["first_class_demand"][demand_index],
        "business_class": demand_data["business_class_demand"][demand_index],
        "economy_class": demand_data["economy_class_demand"][demand_index]
    }
    
    # Track best allocation for this scenario
    best_revenue = 0
    best_revenue_per_class = {"first_class": 0, "business_class": 0, "economy_class": 0}
    
    # Explore allocations within capacity constraints
    for first_class_seats in range(0, total_capacity // int(seat_space["first_class"]) + 1):
        for business_class_seats in range(0, total_capacity // int(seat_space["business_class"]) + 1):
            economy_seats = total_capacity - (
                first_class_seats * seat_space["first_class"] +
                business_class_seats * seat_space["business_class"]
            )
            
            if economy_seats >= 0:
                allocation = {
                    "first_class": first_class_seats,
                    "business_class": business_class_seats,
                    "economy_class": int(economy_seats)
                }
                revenue_per_class, total_allocation_revenue = calculate_revenue_per_class(allocation, demand)
                
                if total_allocation_revenue > best_revenue:
                    best_revenue = total_allocation_revenue
                    best_revenue_per_class = revenue_per_class
    
    # Store the best results for this simulation
    first_class_revenues.append(best_revenue_per_class["first_class"])
    business_class_revenues.append(best_revenue_per_class["business_class"])
    economy_class_revenues.append(best_revenue_per_class["economy_class"])
    total_revenues.append(best_revenue)

# Calculate expected revenue per class and overall
expected_first_class_revenue = np.mean(first_class_revenues)
expected_business_class_revenue = np.mean(business_class_revenues)
expected_economy_class_revenue = np.mean(economy_class_revenues)
expected_total_revenue = np.mean(total_revenues)

print("Expected First Class Revenue:", expected_first_class_revenue)
print("Expected Business Class Revenue:", expected_business_class_revenue)
print("Expected Economy Class Revenue:", expected_economy_class_revenue)
print("Expected Total Revenue:", expected_total_revenue)


Expected First Class Revenue: 24240.96
Expected Business Class Revenue: 39397.38
Expected Economy Class Revenue: 50510.9
Expected Total Revenue: 114149.24


In [3]:
import random
from pulp import LpMaximize, LpProblem, LpVariable, value
import pandas as pd

# Function to simulate demand and solve the optimization problem
def simulate_demand():
    # Randomly select demand scenario
    scenario = random.choices(["High", "Medium", "Low"], weights=[40, 30, 30], k=1)[0]
    
    # Set demand limits based on scenario
    if scenario == "High":
        demand_limits = {"d1": 25, "d2": 60, "d3": 210}
    elif scenario == "Medium":
        demand_limits = {"d1": 12, "d2": 30, "d3": 170}
    else:  # Low demand
        demand_limits = {"d1": 5, "d2": 9, "d3": 150}
    
    # Define the optimization problem
    problem = LpProblem("Maximize_Revenue", LpMaximize)

    # Decision variables
    x1 = LpVariable("First_Class_Seats", lowBound=0, cat="Integer")
    x2 = LpVariable("Business_Class_Seats", lowBound=0, cat="Integer")
    x3 = LpVariable("Economy_Class_Seats", lowBound=0, cat="Integer")
    
    d1 = LpVariable("First_Class_Sold", lowBound=0, cat="Integer")
    d2 = LpVariable("Business_Class_Sold", lowBound=0, cat="Integer")
    d3 = LpVariable("Economy_Class_Sold", lowBound=0, cat="Integer")

    # Objective function: Revenue
    problem += 1600 * d1 + 1100 * d2 + 500 * d3

    # Capacity constraint
    problem += 2 * x1 + 1.5 * x2 + x3 <= 190

    # Demand constraints
    problem += d1 <= demand_limits["d1"]
    problem += d2 <= demand_limits["d2"]
    problem += d3 <= demand_limits["d3"]
    problem += d1 <= x1
    problem += d2 <= x2
    problem += d3 <= x3

    # Solve the problem
    problem.solve()

    # Return the scenario, seat allocations, and revenue
    return {
        "Scenario": scenario,
        "First_Class_Seats": x1.varValue,
        "Business_Class_Seats": x2.varValue,
        "Economy_Class_Seats": x3.varValue,
        "Revenue": value(problem.objective)
    }

# Monte Carlo simulation
num_simulations = 10
results = [simulate_demand() for _ in range(num_simulations)]

# Convert results to a DataFrame for analysis
results_df = pd.DataFrame(results)

# Analyze results
average_revenue = results_df["Revenue"].mean()
max_revenue = results_df["Revenue"].max()
min_revenue = results_df["Revenue"].min()

# Output results
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
simulation_summary = {
    "Average Revenue": f"${average_revenue:,.2f}",
    "Maximum Revenue": f"${max_revenue:,.2f}",
    "Minimum Revenue": f"${min_revenue:,.2f}"
}
results_df, simulation_summary


(  Scenario  First_Class_Seats  Business_Class_Seats  Economy_Class_Seats  \
 0     High               25.0                  60.0                 50.0   
 1     High               25.0                  60.0                 50.0   
 2     High               25.0                  60.0                 50.0   
 3   Medium               12.0                  30.0                121.0   
 4   Medium               12.0                  30.0                121.0   
 5      Low               13.0                   9.0                150.0   
 6     High               25.0                  60.0                 50.0   
 7      Low               13.0                   9.0                150.0   
 8     High               25.0                  60.0                 50.0   
 9      Low               13.0                   9.0                150.0   
 
     Revenue  
 0  131000.0  
 1  131000.0  
 2  131000.0  
 3  112700.0  
 4  112700.0  
 5   92900.0  
 6  131000.0  
 7   92900.0  
 8  131000.0  
 